In [80]:
import pandas as pd
#import hockey_rink as hr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math



In [81]:
# read in the data
df = pd.read_csv('../data/events.csv')


,Period,MatchClock,Timestamp,EventType,TeamStrengthType,TeamStrength,HomeGoalieOnIce,AwayGoalieOnIce,EventPosition,EventStartCoordinate,...,StartPlayerDirection3,StartPlayerDirection4,StartPlayerDirection5,StartPlayerDirection6,StartPlayerDirection7,StartPlayerDirection8,StartPlayerDirection9,StartPlayerDirection10,StartPlayerDirection11,StartPlayerDirection12
0,1,0,1704021018760,Faceoff,FullStrength,5v5,True,True,NeutralZone,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,1704021018760,PuckControl,FullStrength,5v5,True,True,NeutralZone,"-0.5,-0.7",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,1704021020080,PuckControl,FullStrength,5v5,True,True,NeutralZone,"-0.9,-2.9",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,1704021020100,PuckControl,FullStrength,5v5,True,True,NeutralZone,"-1,-2.9",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,1704021020580,BluelineCrossing,FullStrength,5v5,True,True,NeutralZone,"-7.2,-3.2",...,"-0.161,0.987","0.017,-1","-0.612,0.791","-0.864,-0.504","-0.39,0.921","-0.655,0.756","-0.511,-0.859","-0.983,0.183","-0.996,0.085","-0.82,0.572"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4344,3,1198,1704029841680,PuckControl,HomePowerplay,5v4,True,True,HomeTeamZone,"-9.6,-15",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4345,3,1198,1704029842180,BluelineCrossing,HomePowerplay,5v4,True,True,NeutralZone,"-7.2,-14.9",...,"0.39,-0.921","0.809,-0.588","-0.201,-0.98","-0.997,0.081","-0.782,-0.623","-0.814,-0.581","-0.096,-0.995","0.878,-0.479","-0.992,0.13",NaN
4346,3,1198,1704029844920,BluelineCrossing,HomePowerplay,5v4,True,True,NeutralZone,"-7.2,-10.3",...,"0.787,-0.616","0.998,0.064","0.669,0.743","0.003,1","0.044,-0.999","-0.985,-0.173","-0.503,0.864","-0.033,0.999","0.252,-0.968",NaN
4347,3,1198,1704029845080,PuckControl,HomePowerplay,5v4,True,True,HomeTeamZone,"-7.5,-10.5",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# each row has a column EventStartCoordinate and EventEndCoordinate which if applicable, contains the x and y coordinates of the event.
# these columns need to be split into two columns, one for x and one for y
df[['EventStartX', 'EventStartY']] = df['EventStartCoordinate'].str.split(',', expand=True)
df[['EventEndX', 'EventEndY']] = df['EventEndCoordinate'].str.split(',', expand=True)
# convert the columns to numeric
df['EventStartX'] = pd.to_numeric(df['EventStartX'])
df['EventStartY'] = pd.to_numeric(df['EventStartY'])
df['EventEndX'] = pd.to_numeric(df['EventEndX'])
df['EventEndY'] = pd.to_numeric(df['EventEndY'])

df

In [86]:
# Functions

# Euclidean distance between two points
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

,Period,MatchClock,Timestamp,EventType,TeamStrengthType,TeamStrength,HomeGoalieOnIce,AwayGoalieOnIce,EventPosition,EventStartCoordinate,...,StartPlayerDirection3,StartPlayerDirection4,StartPlayerDirection5,StartPlayerDirection6,StartPlayerDirection7,StartPlayerDirection8,StartPlayerDirection9,StartPlayerDirection10,StartPlayerDirection11,StartPlayerDirection12
50,1,49,1704021068840,Shot,FullStrength,5v5,True,True,AwayTeamZone,"9.9,-14.6",...,"-0.023,-1","0.361,-0.933","0.218,-0.976","-0.331,0.944","-0.756,-0.654","-0.471,-0.882","0.272,-0.962","-0.989,0.149","0.356,-0.935","0.56,-0.829"
102,1,92,1704021111720,Shot,FullStrength,5v5,True,True,HomeTeamZone,"-10.1,12.3",...,"-0.838,0.546","-0.575,-0.818","-0.999,-0.047","0.948,0.318","0.831,0.556","0.999,0.042","0.993,0.117","0.923,-0.385","0.873,0.487","0.991,0.13"
156,1,134,1704021182140,Shot,FullStrength,5v5,True,True,AwayTeamZone,"9.4,-12.9",...,"-0.916,-0.401","0.991,-0.133","-0.995,0.097","0.565,-0.825","0.993,-0.121","-0.985,-0.171","-0.969,-0.249","0.877,-0.481","-0.966,-0.259","0.675,0.738"
162,1,137,1704021185520,Shot,FullStrength,5v5,True,True,AwayTeamZone,"25.4,3.8",...,"0.726,0.688","-0.79,0.614","0.927,0.374","-0.985,-0.172","0.564,-0.826","0.427,0.904","0.931,0.366","-0.053,0.999","0.363,0.932","-0.937,0.349"
243,1,206,1704021305800,Shot,FullStrength,5v5,True,True,HomeTeamZone,"-9.5,-8.1",...,"0.973,0.232","-0.345,0.939","0.639,0.769","-0.117,0.993","0.485,0.875","0.982,-0.19","-0.435,0.9","0.544,0.839","0.273,0.962","0.974,0.225"


In [87]:
# just taking the event with type shot or pass
dfshot_goal = df[(df['EventType'] == 'Shot') | (df['EventType'] == 'Pass')]

for index, event in dfshot_goal.head().iterrows():
    
    #print(f"Index: {index}")
    #print(f"Name: {event['EventType']}")

    # Function to calculate the Euclidean distance between two points
    def distance(point1, point2):
        return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
    
    playerNumbers = []

    primary_player_id = event['EventPrimaryPlayerId']
    
    # skip nan values
    if len(df.columns[event == primary_player_id].drop('EventPrimaryPlayerId')) == 0:
        continue
    else:
        primary_player_number = df.columns[event == primary_player_id].drop('EventPrimaryPlayerId')[0][13]
    
    goal_coords = [85,]
    
    if event['EventPrimaryTeam'] == "Home":
        primary_player_team = "Home"
        opponent_team = "Away"
    elif event['EventPrimaryTeam'] == "Away":
        primary_player_team = "Away"
        opponent_team = "Home"
    else:
        print("Error: The team is neither home nor away.")
    
    distances_to_opponents = []
    for i in range(1, 12):

        coords_primary = event[f'StartPlayerCoordinates{primary_player_number}'].split(',')
        float_coords_primary = [float(coord) for coord in coords_primary]
        
        #print(event[f'StartPlayerTeam{i}'])
        if event[f'StartPlayerTeam{i}'] == opponent_team: 
            
            primary_player_id = event['EventPrimaryPlayerId']
            
            #print(df.columns[event == primary_player_id])
           
            coords_player_cur = event[f'StartPlayerCoordinates{i}'].split(',')
            float_coords_player_cur = [float(coord) for coord in coords_player_cur]
            distances_to_opponents.append(distance(float_coords_primary, float_coords_player_cur))

    mean_distance = sum(distances_to_opponents)/len(distances_to_opponents)
    
    print("This is the mean distance of all opponents players to the primary player: " + str(mean_distance))
    
    
    distance_to_goal = 0.0
    
    # determine which goal is correct one


# each row has a column EventStartCoordinate and EventEndCoordinate which if applicable, contains the x and y coordinates of the event.
# these columns need to be split into two columns, one for x and one for y
# df[['EventStartX', 'EventStartY']] = 
# df[['EventEndX', 'EventEndY']] = df['EventEndCoordinate'].str.split(',', expand=True)
# # convert the columns to numeric
# 
# df['EventStartY'] = pd.to_numeric(df['EventStartY'])
# df['EventEndX'] = pd.to_numeric(df['EventEndX'])
# df['EventEndY'] = pd.to_numeric(df['EventEndY'])


xcur = float(df['EventStartCoordinate'].str.split(',')[1])

#    print(event)
    
    if xcur > 0:
        
        distance_to_goal = distance([26.0,0.0], float_coords_player_cur)
    
    else:
        distance_to_goal = distance([-26.0,0.0], float_coords_player_cur)
    
    
    print("This is the distance to the goal: "+ str(distance_to_goal))
    

This is the mean distance of all opponents players to the primary player: 6.9622563150914525
This is the distance to the goal: 2.6400757564888178
This is the mean distance of all opponents players to the primary player: 15.493685045979655
This is the distance to the goal: 46.082751653954
This is the mean distance of all opponents players to the primary player: 14.593205645420378
This is the distance to the goal: 46.24499972970051
This is the mean distance of all opponents players to the primary player: 6.165623320778053
This is the distance to the goal: 53.00339611760741
This is the mean distance of all opponents players to the primary player: 12.478533045950954
This is the distance to the goal: 5.724508712544686


Period                                2
MatchClock                         1000
Timestamp                 1704026154180
EventType                          Shot
TeamStrengthType           FullStrength
                              ...      
StartPlayerDirection8      0.697,-0.717
StartPlayerDirection9      -0.798,0.602
StartPlayerDirection10     0.098,-0.995
StartPlayerDirection11     0.309,-0.951
StartPlayerDirection12     0.598,-0.802
Name: 2670, Length: 114, dtype: object